# Fly a Mission

Run each cell one at a time. Before starting, make sure all three processes are running **in this order**:

1. **Terminal 1** — ArduPilot SITL (start first):  
   ```
   cd ~/ardupilot && Tools/autotest/sim_vehicle.py -v Copter --model JSON -I0 \
     -l -35.3632621,149.1652374,584,0 \
     --out=127.0.0.1:14550 --out=127.0.0.1:14551
   ```
   > **WSL2:** The `--out=127.0.0.1:...` flags are required. Without them MavProxy
   > sends MAVLink to the Windows host IP and nothing reaches the bridge or this notebook.

2. **Terminal 2** — MuJoCo Bridge (start second):  
   `cd hil && python bridge.py --stage 2`  
   Wait until the console prints `AP=connected`.

3. **Terminal 3** — Dummy FC (start third):  
   `cd hil && python dummy_fc.py`  
   Wait until the bridge console shows `FC=ok`.

Then come back here and run the cells below.

## 1. Connect to ArduPilot

In [1]:
from pymavlink import mavutil
import time

# Connect to ArduPilot's secondary GCS port
mav = mavutil.mavlink_connection('udpin:0.0.0.0:14551')

print("Waiting for heartbeat...")
mav.wait_heartbeat()
print(f"Connected to system {mav.target_system}, component {mav.target_component}")

Waiting for heartbeat...


KeyboardInterrupt: 

## 2. Helper functions

These utility functions wrap common MAVLink commands so the later cells stay clean.

In [ ]:
def set_mode(mode_name):
    """Set ArduPilot flight mode by name (e.g. 'GUIDED', 'LAND')."""
    mode_id = mav.mode_mapping().get(mode_name)
    if mode_id is None:
        print(f"Unknown mode: {mode_name}")
        return
    mav.set_mode(mode_id)
    # Wait for mode change ACK
    while True:
        hb = mav.recv_match(type='HEARTBEAT', blocking=True, timeout=3)
        if hb and hb.custom_mode == mode_id:
            print(f"Mode set to {mode_name}")
            return


def arm():
    """Arm the vehicle."""
    mav.mav.command_long_send(
        mav.target_system, mav.target_component,
        mavutil.mavlink.MAV_CMD_COMPONENT_ARM_DISARM,
        0,  # confirmation
        1,  # 1 = arm
        0, 0, 0, 0, 0, 0
    )
    mav.motors_armed_wait()
    print("Armed!")


def takeoff(alt_m):
    """Command takeoff to the given altitude (metres)."""
    mav.mav.command_long_send(
        mav.target_system, mav.target_component,
        mavutil.mavlink.MAV_CMD_NAV_TAKEOFF,
        0,
        0, 0, 0, 0, 0, 0,
        alt_m
    )
    print(f"Takeoff command sent — target {alt_m} m")


def goto_ned(north, east, down, speed=1.0):
    """
    Move to a position relative to home in NED frame.
    north/east/down in metres.
    """
    type_mask = (
        0b0000_1111_1111_1000  # use position only, ignore velocity/accel/yaw
    )
    mav.mav.set_position_target_local_ned_send(
        0,  # time_boot_ms (ignored)
        mav.target_system, mav.target_component,
        mavutil.mavlink.MAV_FRAME_LOCAL_NED,
        type_mask,
        north, east, down,  # position
        0, 0, 0,            # velocity
        0, 0, 0,            # acceleration
        0, 0                # yaw, yaw_rate
    )


def wait_reach(timeout=15):
    """Wait until the vehicle reports it has reached the target (or timeout)."""
    t0 = time.time()
    while time.time() - t0 < timeout:
        pos = mav.recv_match(type='LOCAL_POSITION_NED', blocking=True, timeout=1)
        if pos:
            print(f"  pos: N={pos.x:+.1f}  E={pos.y:+.1f}  D={pos.z:+.1f}", end='\r')
    print()


def land():
    """Switch to LAND mode."""
    set_mode('LAND')
    print("Landing...")


print("Helpers loaded.")

## 3. Set GUIDED mode and Arm

In [ ]:
set_mode('GUIDED')
arm()

## 4. Takeoff to 5 m

Watch the MuJoCo viewer — the drone should lift off.

In [ ]:
takeoff(5)
print("Waiting 10s to reach altitude...")
wait_reach(timeout=10)
print("Takeoff complete.")

## 5. Move Forward (North) 3 m

The drone moves 3 m north while holding 5 m altitude.

In [ ]:
print("Moving forward (north) 3 m...")
goto_ned(north=3, east=0, down=-5)
wait_reach(timeout=10)

## 6. Move Right (East) 3 m

In [ ]:
print("Moving right (east) 3 m...")
goto_ned(north=3, east=3, down=-5)
wait_reach(timeout=10)

## 7. Move Up to 8 m

In [ ]:
print("Moving up to 8 m altitude...")
goto_ned(north=3, east=3, down=-8)
wait_reach(timeout=10)

## 8. Move Left (West) 3 m

In [ ]:
print("Moving left (west) 3 m...")
goto_ned(north=3, east=0, down=-8)
wait_reach(timeout=10)

## 9. Move Back (South) to home X, descend to 5 m

In [ ]:
print("Moving back to above home at 5 m...")
goto_ned(north=0, east=0, down=-5)
wait_reach(timeout=10)

## 10. Land

Watch the MuJoCo viewer — the drone should descend and touch down.

In [ ]:
land()
print("Waiting for touchdown...")
mav.motors_disarmed_wait()
print("Landed and disarmed. Mission complete!")